In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 57.8 MB/s eta 0:00:00


In [ ]:
# const path data wiki
WIKI_PATH = '/content/drive/MyDrive/question_answering_data/wikipedia_20220620_cleaned.jsonl'

In [ ]:
import pandas as pd
import numpy as np
import json
import math
from tqdm import tqdm
import regex as re
import string
tqdm.pandas()

In [ ]:
_WORD_SPLIT = re.compile("([.,!?\"/':;)(])")
_DIGIT_RE = re.compile(br"\d")
STOP_WORDS = "\" \' [ ] . , ! : ; ?".split(" ")

def remove_appending_title(text,title):
    return text.replace(f"{title}\n\n{title}",f"{title} ")

def create_sliding_window(text, size=256, overlap=32):
    actual_size = size - overlap
    windows = []
    n_windows = math.ceil(len(text)/actual_size)
    for i in range(n_windows):
        windows.append(" ".join(text[i*actual_size:i*actual_size + size]))
    return windows

In [ ]:
all_titles = []
all_texts = []
all_bm25_texts = []
with open(WIKI_PATH) as f:
    for i,line in tqdm(enumerate(f)):
        x = json.loads(line)
        text = remove_appending_title(x["text"],x["title"])
        # print(y, "|", text[:50],"\n")
        text = text.split(" ")
        sliding_windows = create_sliding_window(text)
        # bm25_windows = [" ".join(basic_tokenizer(w)) for w in sliding_windows]
        all_texts.extend(sliding_windows)
        # all_bm25_texts.extend(bm25_windows)
        all_titles.extend([x['title'],]*len(sliding_windows))

1273469it [01:28, 14398.88it/s]


In [ ]:
df = pd.DataFrame({"title":all_titles,"text":all_texts})

In [ ]:
df

,title,text
0,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s..."
1,Internet Society,Internet Society hay ISOC là một tổ chức quốc...
2,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi..."
3,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố..."
4,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...
...,...,...
1944401,VNOI,chuyên viên của các tập đoàn công nghệ lớn như...
1944402,Vòng hoa,"Vòng hoa (; IPA: ) là một loại hoa, lá, quả, ..."
1944403,Vòng hoa,vật trong thần thoại La Mã và Hy Lạp được miêu...
1944404,Vòng hoa,"một vai trò nổi bật, tương tự như ở Anh.\n\n==..."


In [ ]:
text = 'Danh sách lỗ đen lớn nhất'
str(df[df['title'] == text]['text'])

'1629973    Danh sách lỗ đen lớn nhất\\n\\nDanh sách các lỗ ...\nName: text, dtype: object'

In [ ]:
# stop_words_file = '/content/drive/MyDrive/question_answering_data/vietnamese-stopwords.txt'
stop_words_file = '/content/drive/MyDrive/question_answering_data/vietnamese-stopwords-dash.txt'
with open(stop_words_file, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

print(stopwords)

['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy', 'ba', 'ba_ba', 'ba_bản', 'ba_cùng', 'ba_họ', 'ba_ngày', 'ba_ngôi', 'ba_tăng', 'bao_giờ', 'bao_lâu', 'bao_nhiêu', 'bao_nả', 'bay_biến', 'biết', 'biết_bao', 'biết_bao_nhiêu', 'biết_chắc', 'biết_chừng_nào', 'biết_mình', 'biết_mấy', 'biết_thế', 'biết_trước', 'biết_việc', 'biết_đâu', 'biết_đâu_chừng', 'biết_đâu_đấy', 'biết_được', 'buổi', 'buổi_làm', 'buổi_mới', 'buổi_ngày', 'buổi_sớm', 'bà', 'bà_ấy', 'bài', 'bài_bác', 'bài_bỏ', 'bài_cái', 'bác', 'bán', 'bán_cấp', 'bán_dạ', 'bán_thế', 'bây_bẩy', 'bây_chừ', 'bây_giờ', 'bây_nhiêu', 'bèn', 'béng', 'bên', 'bên_bị', 'bên_có', 'bên_cạnh', 'bông', 'bước', 'bước_khỏi', 'bước_tới', 'bước_đi', 'bạn', 'bản', 'bản_bộ', 'bản_riêng', 'bản_thân', 'bản_ý', 'bất_chợt', 'bất_cứ', 'bất_giác', 'bất_kì', 'bất_kể', 'bất_kỳ', 'bất_luận', 'bất_ngờ', 'bất_nhược', 'bất_quá', 'bất_quá_chỉ', 'bất_thình_lình', 'bất_tử', 'bất_đồ', 'bấy', 'bấy_chầy', 'bấy_chừ', 'bấy_giờ', 'bấy_lâu', 'bấy_lâ

In [ ]:
from pyvi import ViTokenizer
import re
import string

def basic_tokenizer(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
        # return [w.lower() for w in words if w not in stop_words and w != '' and w != ' ']
    return [w.lower() for w in words if w != '' and w != ' ' and w not in string.punctuation]

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    filtered_text = ' '.join(filtered_words)

    return filtered_text

def process_text(text):
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[{}]+'.format(string.punctuation), '', text) # remove punctuation
    text = ViTokenizer.tokenize(text)
    return text

def post_process(text):
    text = process_text(text)
    text = remove_stopwords(text)

    #text = basic_tokenizer(text)

    return text


In [12]:
# from pandarallel import pandarallel

# pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=12)
df["bm25_text"] = df["text"].progress_apply(lambda x: post_process(x))

 17%|█▋        | 328877/1944406 [1:06:32<5:26:53, 82.37it/s]


KeyboardInterrupt: ignored

In [ ]:
df.to_csv("/content/drive/MyDrive/question_answering_data/processed/wikipedia_20220620_cleaned_v2-1_temp.csv",index=False)

In [ ]:
# str(df.iloc[1]['bm25_text']).replace(f' ', '')

In [ ]:
# texttest = """
# Internet Society  hay ISOC là một tổ chức quốc tế hoạt động phi lợi nhuận, phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những con người cụ thể trong cộng đồng Internet. Mọi chi tiết có thể tìm thấy tại website của ISOC.

# Internet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá nhân. Thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích. Hiện nay tổ chức có tới 90 chi nhánh trên toàn thế giới.

# == Nhiệm vụ và mục đích hoạt động. ==
# Bảo đảm, cổ vũ cho sự phát triển, mở rộng và sử dụng Internet được thuận lợi nhất cho mọi người trên toàn thế giới.

# == Xem thêm. ==
# BULLET::::- Lịch sử Internet

# == Liên kết ngoài. ==
# BULLET::::- ISOC Việt Nam
# BULLET::::- "IETF and the Internet Society" - Về Internet Engineering Task Force và ISOC, bài của Vint Cerf 18/7/1995
# BULLET::::- L’Association Internationale de Lutte Contre la Cybercriminalité bản lưu
# BULLET::::- Public Interest Registry
# """

# post_process(texttest)

# End